<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#A-Trajectory-Analysis-using-the-ICESat2-Satellite" data-toc-modified-id="A-Trajectory-Analysis-using-the-ICESat2-Satellite-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>A Trajectory Analysis using the ICESat2 Satellite</a></span><ul class="toc-item"><li><span><a href="#Step-0:--Pre-processing-the-PCE-data-into-a-G2B-file" data-toc-modified-id="Step-0:--Pre-processing-the-PCE-data-into-a-G2B-file-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Step 0:  Pre-processing the PCE data into a G2B file</a></span><ul class="toc-item"><li><span><a href="#PygeodynPreprocessing" data-toc-modified-id="PygeodynPreprocessing-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>PygeodynPreprocessing</a></span></li></ul></li><li><span><a href="#Step-1:-Run-GEODYN-using-Pygeodyn-with-the-ICESat2-configuration" data-toc-modified-id="Step-1:-Run-GEODYN-using-Pygeodyn-with-the-ICESat2-configuration-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Step 1: Run GEODYN using Pygeodyn with the ICESat2 configuration</a></span><ul class="toc-item"><li><span><a href="#Step-1.1:--Run-with-MSIS2" data-toc-modified-id="Step-1.1:--Run-with-MSIS2-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Step 1.1:  Run with MSIS2</a></span></li><li><span><a href="#Step--1.2:-Run-with-DTM87-for-comparison" data-toc-modified-id="Step--1.2:-Run-with-DTM87-for-comparison-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Step  1.2: Run with DTM87 for comparison</a></span></li></ul></li><li><span><a href="#Read-GEODYN-Output-using-PygeodynRead-functionality" data-toc-modified-id="Read-GEODYN-Output-using-PygeodynRead-functionality-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Read GEODYN Output using PygeodynRead functionality</a></span></li><li><span><a href="#Analyze-output" data-toc-modified-id="Analyze-output-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Analyze output</a></span></li></ul></li></ul></div>

# A Trajectory Analysis using the ICESat2 Satellite

For information on adding the ICESat2 satellite to `pygeodyn` see the notebook titled, *Adding a New Satellite: ICESat2*.



## Step 0:  Pre-processing the PCE data into a G2B file

We will construct a `G2B_PCE`file from a set of binary `.rvg` files which contain trajectory output from a reduced-dynamics run of GEODYN. The files will be stitched together to create a the GEODYN-specific, trajectory-based tracking data type (called PCE (Precice-Clock-Error) ) and stored in a G2B file.

Each `.rvg` file contains a **30-hour arc of ICESat2 trajectory data**.  These datasets are the output from a very precise run of GEODYN in which the orbit has been determined very well (*to a few centimeters accuracy*) using a **reduced dynamics technique** (*empirical accelerations and other parameters are adjusted to account for any mismodelled forces)*.

The process that takes place in `pygeodyn` is as follows:
  1. dump the data from each arc into a usable format
  2. chop of the 3 hour padding on the ends to eliminate discontinuities from end effects
  3. stitch together all the files
  4. smooth over any existing discontinuities between arc gaps or maneuvers.
  5. All data is placed in a single `TRAJ.txt` file which is then fed into a Fortran script (`pce_converer.f`) which converts the data into a `G2B` file to be ingested on `fort.40` of the GEODYN run.

### PygeodynPreprocessing


In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0,'/data/geodyn_proj/pygeodyn/utils_pygeodyn_develop/util_preprocessing/')
from PYGEODYN_Preprocess import PygeodynPreprocessing

path_to_prep_directory = '/data/data_geodyn/inputs/icesat2/pre_processing'
path_to_binaryrvg     = '/data/data_geodyn/inputs/icesat2/pre_processing/traj_files_rvg'

# arc1_files = ['orbit.1807001.2018.287']
arc1_files = ['orbit.1807001.2018.312',
            'orbit.1807001.2018.313',
            'orbit.1807001.2018.314',
            'orbit.1807001.2018.315',
            'orbit.1807001.2018.316',
            'orbit.1807001.2018.317',
            'orbit.1807001.2018.318',
            'orbit.1807001.2018.319',
            'orbit.1807001.2018.320',
            'orbit.1807001.2018.321',
            'orbit.1807001.2018.322',
            'orbit.1807001.2018.323',
            'orbit.1807001.2018.324',
            'orbit.1807001.2018.325',
            'orbit.1807001.2018.326',
            'orbit.1807001.2018.327',
            'orbit.1807001.2018.328',
             ]

##### Uncomment the below call to overwrite the G2B data by re-running the pre-processing code
# Obj = PygeodynPreprocessing(path_to_binaryrvg, path_to_prep_directory,  arc1_files)
# Obj.run_preprocess_PCE()


## Step 1: Run GEODYN using Pygeodyn with the ICESat2 configuration

As a reminder, PYGEODYN is called with the `PYGEODYN` class, but modifications to the ICESAT2 configuration are largely controlled through the `Satellite_ICESat2` class in the `PYGEODYN_Satellites.py` file.   Please refer to this file first for file structure setups and modifications to the Pygeodyn structure.
 
### Step 1.1:  Run with MSIS2


In [ ]:
### Identify which arcs you want to run:
arcs_files = [ 
            '2018.312',
            '2018.313',
            '2018.314',
            '2018.315',
            '2018.316',
            '2018.317',
            '2018.318',
            '2018.319',
            '2018.320',
            '2018.321',
            '2018.322',
            '2018.323',
            '2018.324',
            '2018.325',
            '2018.326',
            '2018.327',
            '2018.328',
             ]

#------ A dictionary containing the run parameters ------  
run_params1 = {} 
run_params1['run_ID']           =  'Run_Arc_0'  
run_params1['arc']              =  'test'
run_params1['satellite']        =  'icesat2'  
run_params1['den_model']        =  'msis2'  
run_params1['empirical_accels'] =  False  
run_params1['SpecialRun_name']  =  ''  
run_params1['options_in']       =  {'DRHODZ_update':True}  
run_params1['verbose']          =  True
run_params1['action']           =  'run'


In [ ]:
%load_ext autoreload
%autoreload 2

sys.path.insert(0, '/data/geodyn_proj/pygeodyn/utils_pygeodyn_develop/')
from PYGEODYN import Pygeodyn



#### Run pyeodyn for the arcs in the above set.
# arc_count = 1
# for run_params1['arc'] in arcs_files:
#     run_params1['run_ID'] = 'Run_Arc_'+str(arccount)
#     print('Running Arc: ', run_params1['arc'])
#     print('Runn ID: '    , run_params1['arc'])

#     Obj_Geodyn = Pygeodyn(run_params1)
#     Obj_Geodyn.RUN_GEODYN()
    
#     arc_count+=1
#     Obj_Geodyn = 0  ## reset the object?




### Step  1.2: Run with DTM87 for comparison


In [ ]:
%load_ext autoreload
%autoreload 2

sys.path.insert(0, '/data/geodyn_proj/pygeodyn/utils_pygeodyn_develop/')
from PYGEODYN import Pygeodyn


import copy
run_params2 = copy.deepcopy(run_params1)

run_params2['SpecialRun_name']   =  ''  
run_params2['den_model']         =  'dtm87'  

#### ------ Run pyeodyn for the arcs in the above set ------
# arc_count = 1
# for run_params2['arc'] in arcs_files:
#     run_params2['run_ID'] = 'Run_Arc_'+str(arc_count)
#     print('Running Arc: ', run_params2['arc'])
#     print('Runn ID: '    , run_params2['arc'])

#     Obj_Geodyn = Pygeodyn(run_params2)
#     Obj_Geodyn.RUN_GEODYN()
    
#     arc_count+=1

    



## Read GEODYN Output using PygeodynRead functionality

### Get MSIS2 Data

In [ ]:
%load_ext autoreload
%autoreload 2

import sys  
sys.path.insert(0, '/data/geodyn_proj/pygeodyn/utils_pygeodyn_develop/')
from PYGEODYN import Pygeodyn

#------ A dictionary containing the run parameters ------  
run_params1 = {} 
run_params1['run_ID']            =  'Run_Arc_1'  
run_params1['arc']               =  [
                                     '2018.312',
                                     '2018.313',
                                     '2018.314',
                                     '2018.315',
                                     '2018.316',
                                     '2018.317'
                                    ] 
run_params1['satellite']         =  'icesat2'  
run_params1['den_model']         =  'msis2'  
run_params1['empirical_accels']  =  False  
run_params1['SpecialRun_name']   =  '_developer_test'  
run_params1['options_in']        =  {'DRHODZ_update':True}  
run_params1['verbose']           =  False
run_params1['action']            =  'read'

Obj_Geodyn1 = Pygeodyn(run_params1)
# Obj_Geodyn1.getData()


### Get DTM87 Data

In [ ]:
import copy
run_params2 = copy.deepcopy(run_params1)

run_params2['SpecialRun_name']   =  ''  
run_params2['den_model']         =  'dtm87'  

Obj_Geodyn2 = Pygeodyn(run_params2)
# Obj_Geodyn2.getData()


## Analyze output
Identify some plot parameters

In [ ]:
import plotly.graph_objects as go
from plotly.offline import plot, iplot
from plotly.subplots import make_subplots
import plotly.express as px


config = dict({
                'displayModeBar': True,
                'responsive': False,
#                 'staticPlot': True,
                'displaylogo': False,
                'showTips': False,
                })

In [ ]:
%load_ext autoreload
%autoreload 2
from PYGEODYNAnalysis_icesat2PCEtrajectory import plot_residuals_observed

fig = make_subplots(rows=3, cols=1, 
            subplot_titles=(['PCE X', 'PCE Y', 'PCE Z']),
            vertical_spacing = 0.1,
                       )

fig = plot_residuals_observed(fig, Obj_Geodyn1, 0)
fig = plot_residuals_observed(fig, Obj_Geodyn2, 1)

# fig.show(config=config)


In [ ]:
%load_ext autoreload
%autoreload 2
from PYGEODYNAnalysis_icesat2PCEtrajectory import plot_cd_and_percdiff_from_apriori


fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=(["Timeseries of Cd", "Percent difference from a priori (Cd=2.2)"]),
    vertical_spacing = 0.08,
    )
fig = plot_cd_and_percdiff_from_apriori(fig, Obj_Geodyn1,  0)
fig = plot_cd_and_percdiff_from_apriori(fig, Obj_Geodyn2,  1)

# fig.show(config=config)

In [ ]:
%load_ext autoreload
%autoreload 2

from PYGEODYNAnalysis_icesat2PCEtrajectory import plot_ScaleDensity_with_CdScaleFactor

fig = make_subplots(rows=2, cols=1,
                subplot_titles=(["Regular (blue) and Scaled (green) Density", "Scaling Factor from Adjusted Cd"]),
                vertical_spacing = 0.08,

                specs=[
                [{"secondary_y": False}],
                [{"secondary_y": True}], ])

fig = plot_ScaleDensity_with_CdScaleFactor(fig, Obj_Geodyn1, 0, 200)
# fig.show(config=config)
